In [150]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from sklearn.preprocessing import RobustScaler
from datetime import datetime, timedelta
from scipy.stats import zscore

In [151]:

def load_and_prepare_data():
    print("LOADING AND PREPARING DATA")
 
    # Load original register data
    df = pd.read_csv('energy_data_dst_cleaned_registers_intact.csv')
    print(f"Original dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
    
    # Convert date column
    df['INTERVAL_DATE'] = pd.to_datetime(df['INTERVAL_DATE'])
    
    # Extract time columns (48 half-hour intervals)
    time_cols = [col for col in df.columns if '.' in col and '-' in col and len(col) == 11]
    print(f"Time columns found: {len(time_cols)} intervals")
    
    # Convert from kWh to kW 
    df[time_cols] = df[time_cols] * 2
    print("Converted time intervals from kWh to kW")
    
    # Load room data
    room_data = pd.read_csv('uniflats.csv')
    room_data.columns = ['Number', 'Street', 'ICP', 'Rooms', 'Corresponding', 'List']
    room_data = room_data[['ICP', 'Rooms']].dropna()
    room_data['ICP'] = room_data['ICP'].astype(str)
    print(f"Room data loaded: {len(room_data)} houses")
    
    # Merge room data
    df = df.merge(room_data, left_on='ICP_IDENTIFIER', right_on='ICP', how='left')
    df = df.drop(columns=['ICP'] if 'ICP' in df.columns else [])

    
    print(f"Data preparation complete:")
    print(f"Date range: {df['INTERVAL_DATE'].min()} to {df['INTERVAL_DATE'].max()}")
    print(f"Unique houses: {df['ICP_IDENTIFIER'].nunique()}")
    print(f"Register types: {df['REGISTER_CONTENTS'].nunique()}")
    
    return df, time_cols

In [152]:
def load_feature_data():
    print("LOADING FEATURE DATASET")

    feature_df = pd.read_csv('final_features.csv')
    
    # ICP_IDENTIFIER as index
    if 'ICP_IDENTIFIER' in feature_df.columns:
        feature_df = feature_df.set_index('ICP_IDENTIFIER')
    
    print(f"Feature dataset loaded: {feature_df.shape[0]} houses, {feature_df.shape[1]} features")
    print(f"Features available: {list(feature_df.columns)}")
    
    return feature_df

In [153]:
def ml_anomaly_detection_on_features(feature_df):
    print("METHOD 1: ML ANOMALY DETECTION ON ENGINEERED FEATURES")

    # Select numerical features for ML
    numerical_features = feature_df.select_dtypes(include=[np.number]).columns.tolist()
    X = feature_df[numerical_features].copy()
    
    # Handle infinite values and missing data
    X = X.replace([np.inf, -np.inf], np.nan)
    X = X.fillna(X.median())
    
    print(f"Using {len(numerical_features)} numerical features")
    print(f"Sample features: {numerical_features[:5]}{'...' if len(numerical_features) > 5 else ''}")
    
    # Robust scaling for anomaly detection 
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Results storage
    feature_ml_results = {}
    contamination_rates = [0.05, 0.10, 0.15, 0.20]
    
    print(f"\nRunning ML algorithms with contamination rates: {[f'{c*100:.0f}%' for c in contamination_rates]}")
    
    # 1. Isolation Forest
    print("\n1. ISOLATION FOREST")
    for contamination in contamination_rates:
        iso_forest = IsolationForest(
            contamination=contamination, 
            random_state=42, 
            n_estimators=100,
            n_jobs=-1
        )
        predictions = iso_forest.fit_predict(X_scaled)
        scores = iso_forest.decision_function(X_scaled)
        
        anomalies = X.index[predictions == -1].tolist()
        feature_ml_results[f'isolation_forest_{contamination}'] = {
            'anomalies': anomalies,
            'scores': scores,
            'count': len(anomalies),
            'method': 'Isolation Forest',
            'contamination': contamination
        }
        print(f"  {contamination*100:4.0f}% contamination: {len(anomalies):4d} anomalies")
    
    # 2. Local Outlier Factor
    print("\n2. LOCAL OUTLIER FACTOR")
    for contamination in contamination_rates:
        lof = LocalOutlierFactor(
            contamination=contamination, 
            n_neighbors=min(20, len(X)//3),
            n_jobs=-1
        )
        predictions = lof.fit_predict(X_scaled)
        scores = lof.negative_outlier_factor_
        
        anomalies = X.index[predictions == -1].tolist()
        feature_ml_results[f'lof_{contamination}'] = {
            'anomalies': anomalies,
            'scores': scores,
            'count': len(anomalies),
            'method': 'Local Outlier Factor',
            'contamination': contamination
        }
        print(f"  {contamination*100:4.0f}% contamination: {len(anomalies):4d} anomalies")
    
    # 3. Elliptic Envelope (Robust Covariance)
    print("\n3. ELLIPTIC ENVELOPE")
    for contamination in contamination_rates:
        try:
            envelope = EllipticEnvelope(
                contamination=contamination, 
                random_state=42,
                support_fraction=None
            )
            predictions = envelope.fit_predict(X_scaled)
            scores = envelope.decision_function(X_scaled)
            
            anomalies = X.index[predictions == -1].tolist()
            feature_ml_results[f'elliptic_envelope_{contamination}'] = {
                'anomalies': anomalies,
                'scores': scores,
                'count': len(anomalies),
                'method': 'Elliptic Envelope',
                'contamination': contamination
            }
            print(f"  {contamination*100:4.0f}% contamination: {len(anomalies):4d} anomalies")
        except Exception as e:
            print(f"  {contamination*100:4.0f}% contamination: FAILED ({str(e)[:50]})")
    
    # 4. One-Class SVM
    print("\n4. ONE-CLASS SVM")
    for contamination in contamination_rates:
        try:
            svm = OneClassSVM(
                nu=contamination, 
                gamma='scale',
                kernel='rbf'
            )
            predictions = svm.fit_predict(X_scaled)
            scores = svm.decision_function(X_scaled)
            
            anomalies = X.index[predictions == -1].tolist()
            feature_ml_results[f'oneclass_svm_{contamination}'] = {
                'anomalies': anomalies,
                'scores': scores,
                'count': len(anomalies),
                'method': 'One-Class SVM',
                'contamination': contamination
            }
            print(f"  {contamination*100:4.0f}% contamination: {len(anomalies):4d} anomalies")
        except Exception as e:
            print(f"  {contamination*100:4.0f}% contamination: FAILED ({str(e)[:50]})")
    
    print(f"\nFeature-based ML detection complete: {len(feature_ml_results)} results")
    return feature_ml_results

In [154]:
def prepare_time_series_data(df, time_cols):
    print("PREPARING TIME SERIES DATA FOR ML")
    
    house_patterns = {}
    
    for icp in df['ICP_IDENTIFIER'].unique():
        house_data = df[df['ICP_IDENTIFIER'] == icp]
        
        # Average pattern across all registers and all days for this house
        avg_pattern = house_data[time_cols].mean(axis=0)
        
        # Only include houses with valid data
        if not avg_pattern.isna().all() and avg_pattern.sum() > 0:
            house_patterns[icp] = avg_pattern.values
    
    # Convert to DataFrame with houses as rows, time intervals as columns
    pattern_df = pd.DataFrame(house_patterns).T
    pattern_df.columns = time_cols
    
    print(f"Time series patterns prepared: {pattern_df.shape[0]} houses, {pattern_df.shape[1]} time intervals")
    print(f"Sample time columns: {time_cols[:3]}...{time_cols[-3:]}")
    
    return pattern_df

In [155]:
def ml_anomaly_detection_on_time_series(pattern_df):
    print("METHOD 2: ML ANOMALY DETECTION ON TIME SERIES DATA")
    
    # Prepare data
    X = pattern_df.copy()
    X = X.fillna(X.mean())  # Fill any missing values with column means
    
    print(f"Time series data shape: {X.shape[0]} houses × {X.shape[1]} time intervals")
    print(f"Data range: {X.min().min():.3f} to {X.max().max():.3f} kW")
    
    # Robust scaling for anomaly detection (better than StandardScaler with outliers)
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Results storage
    timeseries_ml_results = {}
    contamination_rates = [0.05, 0.10, 0.15, 0.20]
    
    print(f"\nRunning ML algorithms with contamination rates: {[f'{c*100:.0f}%' for c in contamination_rates]}")
    
    # 1. Isolation Forest
    print("\n1. ISOLATION FOREST")
    for contamination in contamination_rates:
        iso_forest = IsolationForest(
            contamination=contamination, 
            random_state=42, 
            n_estimators=100,
            n_jobs=-1
        )
        predictions = iso_forest.fit_predict(X_scaled)
        scores = iso_forest.decision_function(X_scaled)
        
        anomalies = X.index[predictions == -1].tolist()
        timeseries_ml_results[f'isolation_forest_{contamination}'] = {
            'anomalies': anomalies,
            'scores': scores,
            'count': len(anomalies),
            'method': 'Isolation Forest',
            'contamination': contamination
        }
        print(f"  {contamination*100:4.0f}% contamination: {len(anomalies):4d} anomalies")
    
    # 2. Local Outlier Factor
    print("\n2. LOCAL OUTLIER FACTOR")
    for contamination in contamination_rates:
        lof = LocalOutlierFactor(
            contamination=contamination, 
            n_neighbors=min(20, len(X)//3),
            n_jobs=-1
        )
        predictions = lof.fit_predict(X_scaled)
        scores = lof.negative_outlier_factor_
        
        anomalies = X.index[predictions == -1].tolist()
        timeseries_ml_results[f'lof_{contamination}'] = {
            'anomalies': anomalies,
            'scores': scores,
            'count': len(anomalies),
            'method': 'Local Outlier Factor',
            'contamination': contamination
        }
        print(f"  {contamination*100:4.0f}% contamination: {len(anomalies):4d} anomalies")
    
    # 3. One-Class SVM
    print("\n3. ONE-CLASS SVM")
    for contamination in contamination_rates:
        try:
            svm = OneClassSVM(
                nu=contamination, 
                gamma='scale',
                kernel='rbf'
            )
            predictions = svm.fit_predict(X_scaled)
            scores = svm.decision_function(X_scaled)
            
            anomalies = X.index[predictions == -1].tolist()
            timeseries_ml_results[f'oneclass_svm_{contamination}'] = {
                'anomalies': anomalies,
                'scores': scores,
                'count': len(anomalies),
                'method': 'One-Class SVM',
                'contamination': contamination
            }
            print(f"  {contamination*100:4.0f}% contamination: {len(anomalies):4d} anomalies")
        except Exception as e:
            print(f"  {contamination*100:4.0f}% contamination: FAILED ({str(e)[:50]})")
    
    # 4. Elliptic Envelope (if computational feasible)
    print("\n4. ELLIPTIC ENVELOPE")
    for contamination in contamination_rates:
        try:
            envelope = EllipticEnvelope(
                contamination=contamination, 
                random_state=42,
                support_fraction=None
            )
            predictions = envelope.fit_predict(X_scaled)
            scores = envelope.decision_function(X_scaled)
            
            anomalies = X.index[predictions == -1].tolist()
            timeseries_ml_results[f'elliptic_envelope_{contamination}'] = {
                'anomalies': anomalies,
                'scores': scores,
                'count': len(anomalies),
                'method': 'Elliptic Envelope',
                'contamination': contamination
            }
            print(f"  {contamination*100:4.0f}% contamination: {len(anomalies):4d} anomalies")
        except Exception as e:
            print(f"  {contamination*100:4.0f}% contamination: FAILED ({str(e)[:50]})")
    
    print(f"\nTime series ML detection complete: {len(timeseries_ml_results)} results")
    return timeseries_ml_results


In [156]:
def compare_ml_approaches(feature_ml_results, timeseries_ml_results, contamination=0.05):
    print(f"COMPARING ML APPROACHES (Using {contamination*100:.0f}% contamination)")
    
    # Get methods that exist in both approaches
    feature_methods = set([k.split('_')[0] + '_' + k.split('_')[1] for k in feature_ml_results.keys() 
                          if f'_{contamination}' in k])
    timeseries_methods = set([k.split('_')[0] + '_' + k.split('_')[1] for k in timeseries_ml_results.keys() 
                             if f'_{contamination}' in k])
    common_methods = feature_methods.intersection(timeseries_methods)
    
    print(f"Common methods available: {list(common_methods)}")
    
    comparison_results = {}
    
    for method in common_methods:
        method_key = f'{method}_{contamination}'
        
        if method_key in feature_ml_results and method_key in timeseries_ml_results:
            feature_anomalies = set(feature_ml_results[method_key]['anomalies'])
            timeseries_anomalies = set(timeseries_ml_results[method_key]['anomalies'])
            
            overlap = feature_anomalies.intersection(timeseries_anomalies)
            feature_only = feature_anomalies - timeseries_anomalies
            timeseries_only = timeseries_anomalies - feature_anomalies
            
            comparison_results[method] = {
                'feature_count': len(feature_anomalies),
                'timeseries_count': len(timeseries_anomalies),
                'overlap_count': len(overlap),
                'feature_only_count': len(feature_only),
                'timeseries_only_count': len(timeseries_only),
                'overlap_houses': list(overlap),
                'feature_only_houses': list(feature_only),
                'timeseries_only_houses': list(timeseries_only)
            }
            
            print(f"\n{method.upper().replace('_', ' ')} Results:")
            print(f"  Feature-based:    {len(feature_anomalies):4d} anomalies")
            print(f"  Time series:      {len(timeseries_anomalies):4d} anomalies")
            print(f"  Both methods:     {len(overlap):4d} houses")
            print(f"  Feature only:     {len(feature_only):4d} houses")
            print(f"  Time series only: {len(timeseries_only):4d} houses")
            
            if len(overlap) > 0:
                overlap_pct = len(overlap) / len(feature_anomalies.union(timeseries_anomalies)) * 100
                print(f"  Overlap rate:     {overlap_pct:4.1f}%")
    
    return comparison_results

In [157]:
def run_dual_ml_detection():
    print("STARTING DUAL ML ANOMALY DETECTION")
    
    # Load data
    df, time_cols = load_and_prepare_data()
    feature_df = load_feature_data()
    
    # Method 1: Feature-based ML
    feature_ml_results = ml_anomaly_detection_on_features(feature_df)
    
    # Method 2: Time series ML
    pattern_df = prepare_time_series_data(df, time_cols)
    timeseries_ml_results = ml_anomaly_detection_on_time_series(pattern_df)
    
    # Compare approaches
    comparison_results = compare_ml_approaches(feature_ml_results, timeseries_ml_results)

    print("DUAL ML DETECTION COMPLETE")
    
    return {
        'feature_ml_results': feature_ml_results,
        'timeseries_ml_results': timeseries_ml_results,
        'comparison_results': comparison_results,
        'feature_df': feature_df,
        'pattern_df': pattern_df,
        'df': df,
        'time_cols': time_cols
    }


In [158]:
results = run_dual_ml_detection()

STARTING DUAL ML ANOMALY DETECTION
LOADING AND PREPARING DATA
Original dataset loaded: 148083 rows, 52 columns
Time columns found: 48 intervals
Converted time intervals from kWh to kW
Room data loaded: 128 houses
Data preparation complete:
Date range: 2022-01-01 00:00:00 to 2024-09-02 00:00:00
Unique houses: 118
Register types: 7
LOADING FEATURE DATASET
Feature dataset loaded: 118 houses, 25 features
Features available: ['num_rooms', 'baseload_kW', 'peak_kW', 'avg_power_kW', 'median_power_kW', 'std_power_kW', 'peak_to_avg_ratio', 'load_factor', 'coefficient_of_variation', 'temperature_correlation', 'heating_sensitivity_kWh', 'seasonal_range_kWh', 'cold_days_avg_kWh', 'weather_analysis_days', 'night_usage_ratio', 'morning_usage_ratio', 'daytime_usage_ratio', 'evening_usage_ratio', 'daily_usage_consistency', 'weekday_weekend_ratio', 'usage_concentration', 'time_analysis_days', 'avg_power_per_room', 'peak_power_per_room', 'power_variability_per_room']
METHOD 1: ML ANOMALY DETECTION ON ENG

/opt/anaconda3/lib/python3.12/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(


Time series patterns prepared: 118 houses, 48 time intervals
Sample time columns: ['00.00-00.30', '00.30-01.00', '01.00-01.30']...['22.30-23.00', '23.00-23.30', '23.30-24.00']
METHOD 2: ML ANOMALY DETECTION ON TIME SERIES DATA
Time series data shape: 118 houses × 48 time intervals
Data range: 0.016 to 24.317 kW

Running ML algorithms with contamination rates: ['5%', '10%', '15%', '20%']

1. ISOLATION FOREST
     5% contamination:    6 anomalies
    10% contamination:   12 anomalies
    15% contamination:   18 anomalies
    20% contamination:   24 anomalies

2. LOCAL OUTLIER FACTOR
     5% contamination:    6 anomalies
    10% contamination:   12 anomalies
    15% contamination:   18 anomalies
    20% contamination:   24 anomalies

3. ONE-CLASS SVM
     5% contamination:    5 anomalies
    10% contamination:   13 anomalies
    15% contamination:   18 anomalies
    20% contamination:   24 anomalies

4. ELLIPTIC ENVELOPE
     5% contamination:    6 anomalies
    10% contamination:   12 an

In [159]:

def statistical_anomaly_detection(feature_df, z_thresholds=[2.0, 2.5, 3.0], iqr_multiplier=1.5):
    print("METHOD 3: STATISTICAL ANOMALY DETECTION")
    
    # Get numerical features
    numerical_features = feature_df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Handle infinite values and missing data
    clean_df = feature_df[numerical_features].copy()
    clean_df = clean_df.replace([np.inf, -np.inf], np.nan)
    clean_df = clean_df.fillna(clean_df.median())
    
    print(f"Analyzing {len(numerical_features)} features for {len(clean_df)} houses")
    print(f"Statistical thresholds: Z-scores {z_thresholds}, IQR multiplier = {iqr_multiplier}")
    
    # Storage for results
    statistical_results = {}
    all_anomalies = set()
    
    # 1. Z-SCORE BASED DETECTION (Multiple Thresholds)
    print(f"\n1. Z-SCORE ANALYSIS (thresholds = {z_thresholds})")
    
    zscore_results = {}
    
    for z_threshold in z_thresholds:
        print(f"\n  Z-Score Threshold = {z_threshold}")
        
        zscore_anomalies = {}
        threshold_houses = set()
        
        # Calculate z-scores for each feature
        for feature in numerical_features:
            z_scores = np.abs(zscore(clean_df[feature]))
            feature_anomalies = clean_df.index[z_scores > z_threshold].tolist()
            
            if len(feature_anomalies) > 0:
                zscore_anomalies[feature] = {
                    'anomalies': feature_anomalies,
                    'z_scores': z_scores[z_scores > z_threshold],
                    'count': len(feature_anomalies)
                }
                print(f"    {feature:30s}: {len(feature_anomalies):2d} anomalies")
                threshold_houses.update(feature_anomalies)
                all_anomalies.update(feature_anomalies)
        
        zscore_results[f'z_{z_threshold}'] = {
            'method': f'Z-Score (threshold = {z_threshold})',
            'anomalies': list(threshold_houses),
            'count': len(threshold_houses),
            'feature_breakdown': zscore_anomalies,
            'threshold': z_threshold
        }
        
        print(f"    TOTAL Z>{z_threshold} ANOMALIES: {len(threshold_houses)} houses")
    
    statistical_results['zscore'] = zscore_results
    
    # 2. IQR (INTERQUARTILE RANGE) BASED DETECTION
    print(f"\n2. IQR ANALYSIS (multiplier = {iqr_multiplier})")
    
    iqr_anomalies = {}
    iqr_houses = set()
    
    # Calculate IQR outliers for each feature
    for feature in numerical_features:
        Q1 = clean_df[feature].quantile(0.25)
        Q3 = clean_df[feature].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - iqr_multiplier * IQR
        upper_bound = Q3 + iqr_multiplier * IQR
        
        # Find outliers
        outliers_mask = (clean_df[feature] < lower_bound) | (clean_df[feature] > upper_bound)
        feature_anomalies = clean_df.index[outliers_mask].tolist()
        
        if len(feature_anomalies) > 0:
            iqr_anomalies[feature] = {
                'anomalies': feature_anomalies,
                'count': len(feature_anomalies),
                'lower_bound': lower_bound,
                'upper_bound': upper_bound,
                'Q1': Q1,
                'Q3': Q3,
                'IQR': IQR
            }
            print(f"  {feature:30s}: {len(feature_anomalies):2d} anomalies (< {lower_bound:.3f} or > {upper_bound:.3f})")
            iqr_houses.update(feature_anomalies)
            all_anomalies.update(feature_anomalies)
    
    statistical_results['iqr'] = {
        'method': 'IQR Analysis',
        'anomalies': list(iqr_houses),
        'count': len(iqr_houses),
        'feature_breakdown': iqr_anomalies,
        'multiplier': iqr_multiplier
    }
    
    print(f"  TOTAL IQR ANOMALIES: {len(iqr_houses)} houses")
    
    # 3. PERCENTILE BASED DETECTION
    print(f"\n3. PERCENTILE ANALYSIS")
    
    percentile_thresholds = [95, 99, 99.5]  # Top percentiles to check
    percentile_anomalies = {}
    
    for percentile in percentile_thresholds:
        percentile_houses = set()
        feature_breakdown = {}
        
        for feature in numerical_features:
            threshold = np.percentile(clean_df[feature], percentile)
            feature_anomalies = clean_df.index[clean_df[feature] > threshold].tolist()
            
            if len(feature_anomalies) > 0:
                feature_breakdown[feature] = {
                    'anomalies': feature_anomalies,
                    'count': len(feature_anomalies),
                    'threshold': threshold
                }
                percentile_houses.update(feature_anomalies)
        
        percentile_anomalies[f'p{percentile}'] = {
            'method': f'{percentile}th Percentile',
            'anomalies': list(percentile_houses),
            'count': len(percentile_houses),
            'feature_breakdown': feature_breakdown,
            'percentile': percentile
        }
        
        print(f"  {percentile:4.1f}th percentile: {len(percentile_houses):2d} houses")
        # ONLY add P99 to all_anomalies (not P95 or P99.5)
        if percentile == 99:
            all_anomalies.update(percentile_houses)
    
    statistical_results['percentile'] = percentile_anomalies

    # 4. MODIFIED Z-SCORE (ROBUST)
    print(f"\n4. MODIFIED Z-SCORE (ROBUST)")
    
    mad_threshold = 3.5  # Modified z-score threshold
    mad_anomalies = {}
    mad_houses = set()
    
    for feature in numerical_features:
        # Calculate Modified Z-Score using MAD (Median Absolute Deviation)
        median = np.median(clean_df[feature])
        mad = np.median(np.abs(clean_df[feature] - median))
        
        if mad != 0:  # Avoid division by zero
            modified_z_scores = 0.6745 * (clean_df[feature] - median) / mad
            modified_z_scores = np.abs(modified_z_scores)
            
            feature_anomalies = clean_df.index[modified_z_scores > mad_threshold].tolist()
            
            if len(feature_anomalies) > 0:
                mad_anomalies[feature] = {
                    'anomalies': feature_anomalies,
                    'modified_z_scores': modified_z_scores[modified_z_scores > mad_threshold],
                    'count': len(feature_anomalies),
                    'median': median,
                    'mad': mad
                }
                print(f"  {feature:30s}: {len(feature_anomalies):2d} anomalies")
                mad_houses.update(feature_anomalies)
                all_anomalies.update(feature_anomalies)
    
    statistical_results['modified_zscore'] = {
        'method': 'Modified Z-Score (MAD)',
        'anomalies': list(mad_houses),
        'count': len(mad_houses),
        'feature_breakdown': mad_anomalies,
        'threshold': mad_threshold
    }
    
    print(f"  TOTAL MAD ANOMALIES: {len(mad_houses)} houses")
    
    # 5. COMBINE ALL STATISTICAL METHODS
    print("STATISTICAL METHODS SUMMARY")
    
    # Get primary methods for comparison (using Z=2.5 as middle threshold)
    primary_zscore_houses = set(zscore_results['z_2.5']['anomalies']) if 'z_2.5' in zscore_results else set()
    
    method_counts = {
        'Z-Score (2.0)': len(zscore_results.get('z_2.0', {}).get('anomalies', [])),
        'Z-Score (2.5)': len(zscore_results.get('z_2.5', {}).get('anomalies', [])),
        'Z-Score (3.0)': len(zscore_results.get('z_3.0', {}).get('anomalies', [])),
        'IQR': len(iqr_houses),
        'Percentile (99th)': len(percentile_anomalies['p99']['anomalies']),
        'Modified Z-Score': len(mad_houses)
    }
    
    for method, count in method_counts.items():
        print(f"{method:20s}: {count:3d} houses")
    
    # Create master statistical anomaly list with confidence scoring
    all_statistical_anomalies = list(all_anomalies)
    statistical_master_list = []
    
    for house in all_statistical_anomalies:
        methods_detected = []
        confidence_score = 0
        
        # Check each method (using primary thresholds)
        if house in primary_zscore_houses:
            methods_detected.append('Z-SCORE')
            confidence_score += 1
            
        if house in iqr_houses:
            methods_detected.append('IQR')
            confidence_score += 1
            
        if house in percentile_anomalies['p99']['anomalies']:
            methods_detected.append('P99')
            confidence_score += 1
            
        if house in mad_houses:
            methods_detected.append('MAD')
            confidence_score += 1
        
        # Get house characteristics for context
        house_info = {}
        if house in feature_df.index:
            house_info = {
                'rooms': feature_df.loc[house, 'num_rooms'],
                'baseload': feature_df.loc[house, 'baseload_kW'],
                'avg_power': feature_df.loc[house, 'avg_power_kW'],
                'avg_power_per_room': feature_df.loc[house, 'avg_power_per_room'],
                'peak_per_room': feature_df.loc[house, 'peak_power_per_room'],
                'load_factor': feature_df.loc[house, 'load_factor']
            }
        
        statistical_master_list.append({
            'house_id': house,
            'confidence_score': confidence_score,
            'methods_detected': methods_detected,
            'detection_summary': ' + '.join(methods_detected),
            'house_info': house_info
        })
    
    # Sort by confidence score
    statistical_master_list.sort(key=lambda x: x['confidence_score'], reverse=True)
    
    print(f"\nSTATISTICAL MASTER LIST: {len(statistical_master_list)} houses")
    print("Confidence breakdown:")
    for conf in sorted(set(x['confidence_score'] for x in statistical_master_list), reverse=True):
        count = sum(1 for x in statistical_master_list if x['confidence_score'] == conf)
        print(f"  {conf}/4 methods: {count:3d} houses")
    
    # Add to main results
    statistical_results['master_list'] = statistical_master_list
    statistical_results['all_anomalies'] = all_statistical_anomalies
    
    print(f"\nStatistical anomaly detection complete: {len(all_statistical_anomalies)} total anomalies")
    
    return statistical_results

In [160]:
statistical_results = statistical_anomaly_detection(results['feature_df'])

METHOD 3: STATISTICAL ANOMALY DETECTION
Analyzing 25 features for 118 houses
Statistical thresholds: Z-scores [2.0, 2.5, 3.0], IQR multiplier = 1.5

1. Z-SCORE ANALYSIS (thresholds = [2.0, 2.5, 3.0])

  Z-Score Threshold = 2.0
    num_rooms                     :  4 anomalies
    baseload_kW                   :  1 anomalies
    peak_kW                       :  3 anomalies
    avg_power_kW                  :  1 anomalies
    median_power_kW               :  1 anomalies
    std_power_kW                  :  2 anomalies
    peak_to_avg_ratio             :  3 anomalies
    load_factor                   :  5 anomalies
    coefficient_of_variation      :  3 anomalies
    temperature_correlation       :  9 anomalies
    heating_sensitivity_kWh       :  1 anomalies
    seasonal_range_kWh            :  1 anomalies
    cold_days_avg_kWh             :  1 anomalies
    weather_analysis_days         : 11 anomalies
    night_usage_ratio             :  3 anomalies
    morning_usage_ratio           :  5

In [161]:
def pattern_anomaly_detection(df, time_cols, room_data=None):
    print("METHOD 4: BEHAVIOURAL PATTERN ANOMALY DETECTION")
    
    print(f"Analysing behavioural power patterns for {df['ICP_IDENTIFIER'].nunique()} houses")
    print(f"Using {len(time_cols)} time intervals from {df.shape[0]} total records")
    print("Focus: Pure behavioural anomalies (no weekly/seasonal patterns)")
    
    # Load room data if not provided
    if room_data is None and 'Rooms' in df.columns:
        print("Using room data from merged dataset")
        room_info = df[['ICP_IDENTIFIER', 'Rooms']].drop_duplicates().set_index('ICP_IDENTIFIER')['Rooms'].to_dict()
    else:
        print("Room data not available - proceeding without room normalization")
        room_info = {}
    
    print(f"Room information available for {len(room_info)} houses")
    
    # Storage for results
    pattern_results = {}
    all_pattern_anomalies = set()
    
    # BEHAVIORAL PATTERN ANALYSIS (INTRA-DAY USAGE BEHAVIORS)
    print(f"\nBEHAVIORAL POWER PATTERN ANALYSIS")
    print("-" * 50)
    
    behavioral_patterns = {}
    
    for icp in df['ICP_IDENTIFIER'].unique():
        house_data = df[df['ICP_IDENTIFIER'] == icp]
        
        # Average daily pattern across all days and registers
        avg_daily_pattern = house_data[time_cols].mean()
        
        if not avg_daily_pattern.isna().all() and avg_daily_pattern.sum() > 0:
            # Get room count for this house
            room_count = room_info.get(icp) 
            
            # Calculate pattern metrics (all in kW) 
            min_power = avg_daily_pattern.min()       # Minimum power across all intervals
            max_power = avg_daily_pattern.max()       # Maximum power across all intervals (peak)
            avg_power = avg_daily_pattern.mean()      # Average power across all intervals
            
            # Ensure logical order: min <= avg <= max
            assert min_power <= avg_power <= max_power, f"Power logic error for {icp}: min={min_power:.3f}, avg={avg_power:.3f}, max={max_power:.3f}"
            
            # Room-normalised metrics (kW per room)
            avg_power_per_room = avg_power / room_count
            peak_power_per_room = max_power / room_count
            min_power_per_room = min_power / room_count
            
            # Time of peak power (which half-hour interval)
            peak_time_idx = avg_daily_pattern.idxmax()
            # Extract hour from column name 
            try:
                hour_str = peak_time_idx.split('.')[0]
                minute_str = peak_time_idx.split('.')[1].split('-')[0]
                peak_hour = float(hour_str) + float(minute_str)/60
            except:
                peak_hour = 12.0  # Default if parsing fails
            
            # Time-of-use power analysis (kW) - STUDENT-FOCUSED
            # Night: 11 PM - 6 AM (students often stay up late)
            night_hours = [col for col in time_cols if col.startswith(('23.', '00.', '01.', '02.', '03.', '04.', '05.'))]
            # Morning: 6 AM - 10 PM 
            morning_hours = [col for col in time_cols if col.startswith(('06.','07.', '08.', '09.'))]
            # Daytime: 10 PM - 5 PM
            afternoon_hours = [col for col in time_cols if col.startswith(('10.', '11.', '12.', '13.', '14.', '15.', '16.'))]
            # Evening: 6 PM - 11 PM (peak social/cooking time)
            evening_hours = [col for col in time_cols if col.startswith(('17.', '18.', '19.', '20.', '21.', '22.'))]

            night_power = avg_daily_pattern[night_hours].mean() if night_hours else 0
            morning_power = avg_daily_pattern[morning_hours].mean() if morning_hours else 0
            afternoon_power = avg_daily_pattern[afternoon_hours].mean() if afternoon_hours else 0
            evening_power = avg_daily_pattern[evening_hours].mean() if evening_hours else 0
            
            # Power pattern consistency (coefficient of variation)
            power_cv = avg_daily_pattern.std() / avg_daily_pattern.mean() if avg_daily_pattern.mean() > 0 else 0
            
            # Baseload estimation 
            # Method 1: Simple minimum (most conservative)
            baseload_simple = min_power
            
            # Method 2: Bottom 10th percentile average (more robust)
            sorted_intervals = sorted(avg_daily_pattern.values)
            bottom_10pct_count = max(1, len(sorted_intervals) // 10)
            baseload_10pct = np.mean(sorted_intervals[:bottom_10pct_count])
            
            # Choose the most conservative (lowest) baseload estimate
            estimated_baseload = min(baseload_simple, baseload_10pct)
            
            # Ensure baseload is logical: baseload <= min_power <= avg_power <= max_power
            estimated_baseload = min(estimated_baseload, min_power)
            
            # Additional behavioural metrics
            power_range = max_power - min_power
            evening_to_night_ratio = evening_power / night_power if night_power > 0 else 0
            morning_to_afternoon_ratio = morning_power / afternoon_power if afternoon_power > 0 else 0
            
            behavioral_patterns[icp] = {
                'room_count': room_count,
                'min_power': min_power,
                'avg_power': avg_power,
                'peak_power': max_power,
                'power_range': power_range,
                'avg_power_per_room': avg_power_per_room,
                'peak_power_per_room': peak_power_per_room,
                'min_power_per_room': min_power_per_room,
                'peak_hour': peak_hour,
                'night_power': night_power,
                'morning_power': morning_power,
                'afternoon_power': afternoon_power,
                'evening_power': evening_power,
                'night_power_per_room': night_power / room_count,
                'morning_power_per_room': morning_power / room_count,
                'afternoon_power_per_room': afternoon_power / room_count,
                'evening_power_per_room': evening_power / room_count,
                'power_cv': power_cv,
                'estimated_baseload': estimated_baseload,
                'peak_to_baseload_ratio': max_power / estimated_baseload if estimated_baseload > 0 else 0,
                'evening_to_night_ratio': evening_to_night_ratio,
                'morning_to_afternoon_ratio': morning_to_afternoon_ratio,
                'baseload_to_avg_ratio': estimated_baseload / avg_power if avg_power > 0 else 0
            }
    
    # Convert to DataFrame for analysis
    behavioral_df = pd.DataFrame(behavioral_patterns).T
    print(f"Behavioral power patterns calculated for {len(behavioral_df)} houses")
    
    # Find behavioral anomalies using statistical methods 
    behavioral_anomalies = {}
    
    # 1. TIMING ANOMALIES
    # Very early morning peaks (3-6 AM)
    early_morning_peaks = behavioral_df[
        (behavioral_df['peak_hour'] >= 3) & (behavioral_df['peak_hour'] < 6)
    ].index.tolist()
    behavioral_anomalies['early_morning_peaks'] = early_morning_peaks
    print(f"  Very early morning peaks (3-6 AM): {len(early_morning_peaks)} houses")
    
    # Very late night peaks (after midnight)
    late_night_peaks = behavioral_df[behavioral_df['peak_hour'] > 24].index.tolist()
    behavioral_anomalies['late_night_peaks'] = late_night_peaks
    print(f"  Late night peaks (after midnight): {len(late_night_peaks)} houses")
    
    # Unusual evening-to-night ratios 
    evening_night_low = behavioral_df['evening_to_night_ratio'].quantile(0.05)
    evening_night_high = behavioral_df['evening_to_night_ratio'].quantile(0.95)
    unusual_evening_night = behavioral_df[
        (behavioral_df['evening_to_night_ratio'] < evening_night_low) | 
        (behavioral_df['evening_to_night_ratio'] > evening_night_high)
    ].index.tolist()
    behavioral_anomalies['unusual_evening_night_patterns'] = unusual_evening_night
    print(f"  Unusual evening/night patterns (<5th or >95th percentile): {len(unusual_evening_night)} houses")
    
    # 2. LOAD PATTERN ANOMALIES
    # Extremely flat power patterns (very low CV - always-on devices/vacant)
    low_cv_threshold = behavioral_df['power_cv'].quantile(0.05)
    flat_patterns = behavioral_df[behavioral_df['power_cv'] < low_cv_threshold].index.tolist()
    behavioral_anomalies['flat_power_patterns'] = flat_patterns
    print(f"  Extremely flat power patterns (<5th percentile CV): {len(flat_patterns)} houses")
    
    # Extremely variable power patterns (very high CV - erratic usage/multiple occupants)
    high_cv_threshold = behavioral_df['power_cv'].quantile(0.95)
    erratic_patterns = behavioral_df[behavioral_df['power_cv'] > high_cv_threshold].index.tolist()
    behavioral_anomalies['erratic_power_patterns'] = erratic_patterns
    print(f"  Extremely variable power patterns (>95th percentile CV): {len(erratic_patterns)} houses")
    
    # Extreme peak-to-baseload ratios (inefficient devices or unusual loads)
    peak_baseload_threshold = behavioral_df['peak_to_baseload_ratio'].quantile(0.95)
    extreme_peak_ratios = behavioral_df[behavioral_df['peak_to_baseload_ratio'] > peak_baseload_threshold].index.tolist()
    behavioral_anomalies['extreme_peak_ratios'] = extreme_peak_ratios
    print(f"  Extreme peak-to-baseload ratios (>95th percentile): {len(extreme_peak_ratios)} houses")
    
    # 3. CONSUMPTION LEVEL ANOMALIES
    # Very high average power per room (potential overconsumption)
    high_avg_power_per_room_threshold = behavioral_df['avg_power_per_room'].quantile(0.95)
    high_power_per_room = behavioral_df[behavioral_df['avg_power_per_room'] > high_avg_power_per_room_threshold].index.tolist()
    behavioral_anomalies['high_avg_power_per_room'] = high_power_per_room
    print(f"  High average power per room (>95th percentile): {len(high_power_per_room)} houses")
    
    # Very low average power per room (potentially vacant or minimal usage)
    low_avg_power_per_room_threshold = behavioral_df['avg_power_per_room'].quantile(0.05)
    low_power_per_room = behavioral_df[behavioral_df['avg_power_per_room'] < low_avg_power_per_room_threshold].index.tolist()
    behavioral_anomalies['low_avg_power_per_room'] = low_power_per_room
    print(f"  Very low average power per room (<5th percentile): {len(low_power_per_room)} houses")
    
    # 4. BASELOAD ANOMALIES
    # Very high baseload-to-average ratio (always-on heavy devices)
    high_baseload_ratio_threshold = behavioral_df['baseload_to_avg_ratio'].quantile(0.95)
    high_baseload_ratio = behavioral_df[behavioral_df['baseload_to_avg_ratio'] > high_baseload_ratio_threshold].index.tolist()
    behavioral_anomalies['high_baseload_ratio'] = high_baseload_ratio
    print(f"  High baseload-to-average ratio (>95th percentile): {len(high_baseload_ratio)} houses")
    
    # Very low baseload (potentially vacant or unusual setup)
    low_baseload_threshold = behavioral_df['estimated_baseload'].quantile(0.05)
    low_baseload = behavioral_df[behavioral_df['estimated_baseload'] < low_baseload_threshold].index.tolist()
    behavioral_anomalies['low_baseload'] = low_baseload
    print(f"  Very low baseload (<5th percentile): {len(low_baseload)} houses")
    
    #  Very high baseload (always-on inefficient devices)
    high_baseload_threshold = behavioral_df['estimated_baseload'].quantile(0.95)
    high_baseload = behavioral_df[behavioral_df['estimated_baseload'] > high_baseload_threshold].index.tolist()
    behavioral_anomalies['high_baseload'] = high_baseload
    print(f"  Very high baseload (>95th percentile): {len(high_baseload)} houses")
    
    # 5. EXTREME POWER RANGES
    # Very high power range (big difference between min and max)
    high_range_threshold = behavioral_df['power_range'].quantile(0.95)
    high_power_range = behavioral_df[behavioral_df['power_range'] > high_range_threshold].index.tolist()
    behavioral_anomalies['high_power_range'] = high_power_range
    print(f"  High power range (>95th percentile): {len(high_power_range)} houses")
    
    # Very low power range (very consistent usage - potentially vacant or single device)
    low_range_threshold = behavioral_df['power_range'].quantile(0.05)
    low_power_range = behavioral_df[behavioral_df['power_range'] < low_range_threshold].index.tolist()
    behavioral_anomalies['low_power_range'] = low_power_range
    print(f"  Low power range (<5th percentile): {len(low_power_range)} houses")
    
    pattern_results['behavioral_analysis'] = {
        'patterns': behavioral_df,
        'anomalies': behavioral_anomalies
    }
    
    # Add to master list
    for anomaly_list in behavioral_anomalies.values():
        all_pattern_anomalies.update(anomaly_list)
    
    # BEHAVIOURAL ANOMALY SUMMARY
    print("BEHAVIOURAL ANOMALY SUMMARY")
    
    # Count total anomalies
    total_count = len(all_pattern_anomalies)
    
    print(f"Total behavioral anomalies detected: {total_count} houses")
    
    # Create behavioral anomaly list with confidence scoring
    all_pattern_anomalies_list = list(all_pattern_anomalies)
    pattern_master_list = []
    
    for house in all_pattern_anomalies_list:
        # Count how many different anomaly types this house has
        anomaly_count = 0
        detected_types = []
        
        for anomaly_type, house_list in behavioral_anomalies.items():
            if house in house_list:
                anomaly_count += 1
                detected_types.append(anomaly_type)
        
        # Get pattern characteristics for context
        pattern_info = {}
        if house in behavioral_df.index:
            pattern_info.update({
                'room_count': behavioral_df.loc[house, 'room_count'],
                'peak_hour': behavioral_df.loc[house, 'peak_hour'],
                'power_cv': behavioral_df.loc[house, 'power_cv'],
                'evening_to_night_ratio': behavioral_df.loc[house, 'evening_to_night_ratio'],
                'avg_power_per_room': behavioral_df.loc[house, 'avg_power_per_room'],
                'peak_power_per_room': behavioral_df.loc[house, 'peak_power_per_room'],
                'estimated_baseload': behavioral_df.loc[house, 'estimated_baseload'],
                'min_power': behavioral_df.loc[house, 'min_power'],
                'avg_power': behavioral_df.loc[house, 'avg_power'],
                'peak_power': behavioral_df.loc[house, 'peak_power'],
                'baseload_to_avg_ratio': behavioral_df.loc[house, 'baseload_to_avg_ratio'],
                'power_range': behavioral_df.loc[house, 'power_range']
            })
        
        pattern_master_list.append({
            'house_id': house,
            'anomaly_count': anomaly_count,
            'detected_types': detected_types,
            'detection_summary': f"{anomaly_count} anomaly types",
            'pattern_info': pattern_info
        })
    
    # Sort by anomaly count (houses with more anomaly types are more suspicious)
    pattern_master_list.sort(key=lambda x: x['anomaly_count'], reverse=True)
    
    print(f"\nBEHAVIORAL ANOMALY MASTER LIST: {len(pattern_master_list)} houses")
    print("Anomaly type count breakdown:")
    
    anomaly_counts = {}
    for item in pattern_master_list:
        count = item['anomaly_count']
        anomaly_counts[count] = anomaly_counts.get(count, 0) + 1
    
    for count in sorted(anomaly_counts.keys(), reverse=True):
        houses = anomaly_counts[count]
        print(f"  {count} anomaly types: {houses:3d} houses")
    
    # Add to main results
    pattern_results['master_list'] = pattern_master_list
    pattern_results['all_anomalies'] = all_pattern_anomalies_list
    
    print(f"Behavioral pattern anomaly detection complete: {len(all_pattern_anomalies_list)} total anomalies")
    
    return pattern_results


In [162]:
pattern_results = pattern_anomaly_detection(results['df'], results['time_cols'])

METHOD 4: BEHAVIOURAL PATTERN ANOMALY DETECTION
Analysing behavioural power patterns for 118 houses
Using 48 time intervals from 148083 total records
Focus: Pure behavioural anomalies (no weekly/seasonal patterns)
Using room data from merged dataset
Room information available for 118 houses

BEHAVIORAL POWER PATTERN ANALYSIS
--------------------------------------------------
Behavioral power patterns calculated for 118 houses
  Very early morning peaks (3-6 AM): 2 houses
  Late night peaks (after midnight): 0 houses
  Unusual evening/night patterns (<5th or >95th percentile): 12 houses
  Extremely flat power patterns (<5th percentile CV): 6 houses
  Extremely variable power patterns (>95th percentile CV): 6 houses
  Extreme peak-to-baseload ratios (>95th percentile): 6 houses
  High average power per room (>95th percentile): 6 houses
  Very low average power per room (<5th percentile): 6 houses
  High baseload-to-average ratio (>95th percentile): 6 houses
  Very low baseload (<5th perc

In [163]:
def get_all_anomalous_houses():
    # METHOD 1: ML ANOMALY DETECTION 
    
    ml_anomalies = set()
    contamination_target = 0.05
    
    # Feature-based ML results
    for method_key, method_data in results['feature_ml_results'].items():
        if f'_{contamination_target}' in method_key:  # Look for _0.15
            ml_anomalies.update(method_data['anomalies'])
    
    # Time series ML results  
    for method_key, method_data in results['timeseries_ml_results'].items():
        if f'_{contamination_target}' in method_key:  # Look for _0.15
            ml_anomalies.update(method_data['anomalies'])
    
    print(f"METHOD 1 - ML Anomalies (15% contamination): {len(ml_anomalies)} unique houses")

    # METHOD 2: STATISTICAL ANOMALY DETECTION
  
    statistical_anomalies = set(statistical_results['all_anomalies'])
    print(f"METHOD 2 - Statistical Anomalies: {len(statistical_anomalies)} unique houses")
    
    # METHOD 3: BEHAVIORAL PATTERN DETECTION
    
    behavioral_anomalies = set(pattern_results['all_anomalies'])
    print(f"METHOD 3 - Behavioral Pattern Anomalies: {len(behavioral_anomalies)} unique houses")
    
    # COMBINE ALL METHODS - TOTAL UNIQUE HOUSES
    
    all_anomalous_houses = ml_anomalies.union(statistical_anomalies).union(behavioral_anomalies)
    
    print(f"TOTAL UNIQUE ANOMALOUS HOUSES (ALL METHODS): {len(all_anomalous_houses)}")
    
    # Create detailed breakdown
    breakdown = {
        'ml_only': ml_anomalies - statistical_anomalies - behavioral_anomalies,
        'statistical_only': statistical_anomalies - ml_anomalies - behavioral_anomalies,
        'behavioral_only': behavioral_anomalies - ml_anomalies - statistical_anomalies,
        'ml_statistical': (ml_anomalies & statistical_anomalies) - behavioral_anomalies,
        'ml_behavioral': (ml_anomalies & behavioral_anomalies) - statistical_anomalies,
        'statistical_behavioral': (statistical_anomalies & behavioral_anomalies) - ml_anomalies,
        'all_three_methods': ml_anomalies & statistical_anomalies & behavioral_anomalies
    }
    
    print("\nBREAKDOWN BY METHOD OVERLAP:")
    print(f"ML only:                    {len(breakdown['ml_only']):3d} houses")
    print(f"Statistical only:           {len(breakdown['statistical_only']):3d} houses") 
    print(f"Behavioral only:            {len(breakdown['behavioral_only']):3d} houses")
    print(f"ML + Statistical:           {len(breakdown['ml_statistical']):3d} houses")
    print(f"ML + Behavioural:            {len(breakdown['ml_behavioral']):3d} houses")
    print(f"Statistical + Behavioural:   {len(breakdown['statistical_behavioral']):3d} houses")
    print(f"ALL THREE METHODS:          {len(breakdown['all_three_methods']):3d} houses")
    
    # CROSS-METHOD VALIDATION: Statistical + (ML OR Behavioral)  
    print("APPLYING CROSS-METHOD VALIDATION")
    print("Requiring: Statistical Detection + (ML OR Behavioural confirmation)")
    
    # Apply cross-validation filter
    validated_anomalies = set()
    
    for house in all_anomalous_houses:
        is_statistical = house in statistical_anomalies
        is_ml = house in ml_anomalies  
        is_behavioral = house in behavioral_anomalies
        
        # Cross-method validation: Statistical + at least one other method
        if is_statistical and (is_ml or is_behavioral):
            validated_anomalies.add(house)
    
    print(f"Houses passing cross-validation: {len(validated_anomalies)}")
    print(f"Reduction: {len(all_anomalous_houses)} → {len(validated_anomalies)} houses")
    
    # Create validated breakdown
    validated_breakdown = {
        'ml_statistical': (ml_anomalies & statistical_anomalies) - behavioral_anomalies,
        'statistical_behavioral': (statistical_anomalies & behavioral_anomalies) - ml_anomalies,
        'all_three_methods': ml_anomalies & statistical_anomalies & behavioral_anomalies
    }
    
    print("\nVALIDATED BREAKDOWN:")
    print(f"Statistical + ML:           {len(validated_breakdown['ml_statistical']):3d} houses")
    print(f"Statistical + Behavioral:   {len(validated_breakdown['statistical_behavioral']):3d} houses")
    print(f"ALL THREE METHODS:          {len(validated_breakdown['all_three_methods']):3d} houses")
    
   # Return everything for further analysis
    return {
        'all_anomalous_houses': sorted(list(all_anomalous_houses)),
        'validated_anomalies': sorted(list(validated_anomalies)),  
        'ml_anomalies': sorted(list(ml_anomalies)),
        'statistical_anomalies': sorted(list(statistical_anomalies)),
        'behavioral_anomalies': sorted(list(behavioral_anomalies)),
        'breakdown': breakdown,
        'validated_breakdown': validated_breakdown, 
        'total_count': len(all_anomalous_houses),
        'validated_count': len(validated_anomalies)  
    }


In [164]:
all_houses_analysis = get_all_anomalous_houses()

METHOD 1 - ML Anomalies (15% contamination): 17 unique houses
METHOD 2 - Statistical Anomalies: 60 unique houses
METHOD 3 - Behavioral Pattern Anomalies: 34 unique houses
TOTAL UNIQUE ANOMALOUS HOUSES (ALL METHODS): 64

BREAKDOWN BY METHOD OVERLAP:
ML only:                      0 houses
Statistical only:            27 houses
Behavioral only:              4 houses
ML + Statistical:             3 houses
ML + Behavioural:              0 houses
Statistical + Behavioural:    16 houses
ALL THREE METHODS:           14 houses
APPLYING CROSS-METHOD VALIDATION
Requiring: Statistical Detection + (ML OR Behavioural confirmation)
Houses passing cross-validation: 33
Reduction: 64 → 33 houses

VALIDATED BREAKDOWN:
Statistical + ML:             3 houses
Statistical + Behavioral:    16 houses
ALL THREE METHODS:           14 houses


In [165]:
# Simple clustering validation
clustering_anomalies = pd.read_csv('extreme_outliers.csv')
clustering_houses = set(clustering_anomalies['ICP_IDENTIFIER'].tolist())
our_anomalies = set(all_houses_analysis['all_anomalous_houses'])

# Check if clustering houses are in our list
overlap = clustering_houses.intersection(our_anomalies)
new_houses = clustering_houses - our_anomalies

# Results
if len(overlap) > 0:
    print(f"Clustering detected outliers are indeed anomalies: {list(overlap)}")

if len(new_houses) > 0:
    print(f" Added houses clustering detected as anomaly: {list(new_houses)}")
    # Add to our list
    updated_houses = all_houses_analysis['all_anomalous_houses'] + list(new_houses)
    all_houses_analysis['all_anomalous_houses'] = updated_houses
    all_houses_analysis['total_count'] = len(updated_houses)
    print(f" Total anomalous houses now: {len(updated_houses)}")

print("Clustering validation complete!")

Clustering detected outliers are indeed anomalies: ['0000002867DED9E', '0000003664DE6F5']
Clustering validation complete!


In [166]:
def setup_appliance_detection():
    print("REGISTER-BASED APPLIANCE IDENTIFICATION SETUP")

    # Check unique registers
    print(f"\nUnique register codes:")
    unique_registers = results['df']['REGISTER_CONTENTS'].unique()
    print(f"Found {len(unique_registers)} unique register codes:")
    for register in sorted(unique_registers):
        print(f"  {register}")
    
    # Check houses with multiple registers
    print(f"\nHouses with multiple registers:")
    house_register_counts = results['df'].groupby('ICP_IDENTIFIER')['REGISTER_CONTENTS'].nunique()
    multi_register_houses = house_register_counts[house_register_counts > 1]
    
    if len(multi_register_houses) > 0:
        print(f"Found {len(multi_register_houses)} houses with multiple register types:")
        for house in multi_register_houses.index:
            registers = results['df'][results['df']['ICP_IDENTIFIER'] == house]['REGISTER_CONTENTS'].unique()
            print(f"  {house}: {list(registers)}")
    else:
        print("All houses have single register type")
    
    # Separate by register type for appliance identification
    print(f"\nSeparating by Register Type:")
    register_separation = {}
    
    for register in unique_registers:
        houses_with_register = results['df'][results['df']['REGISTER_CONTENTS'] == register]['ICP_IDENTIFIER'].unique()
        register_separation[register] = {
            'houses': houses_with_register,
            'house_count': len(houses_with_register),
            'data': results['df'][results['df']['REGISTER_CONTENTS'] == register]
        }
        print(f"  {register}: {len(houses_with_register)} houses")
    
    return register_separation, unique_registers


In [167]:
register_data, unique_registers = setup_appliance_detection()

REGISTER-BASED APPLIANCE IDENTIFICATION SETUP

Unique register codes:
Found 7 unique register codes:
  CN11
  CN16
  CN8
  IN16
  IN19
  N8|D16
  UN24

Houses with multiple registers:
Found 30 houses with multiple register types:
  0000002807DE26E: ['CN8', 'IN16']
  0000002810DE509: ['CN8', 'IN19']
  0000002811DE94C: ['CN11', 'IN19']
  0000003255DE149: ['CN11', 'IN19']
  0000003265DE6B1: ['CN8', 'IN19']
  0000003949DEBF4: ['CN8', 'IN19']
  0000100004DE98F: ['CN8', 'IN19']
  0000100008DEA91: ['CN8', 'IN19']
  0000100028DE7C4: ['CN8', 'IN16']
  0000100046DEBAF: ['CN8', 'IN19']
  0000100047DE7EA: ['CN8', 'IN19']
  0000101705DEB68: ['CN8', 'IN19']
  0000201071DEC89: ['CN8', 'IN19']
  0000201073DEC0C: ['CN8', 'IN16']
  0000201074DE1C6: ['CN8', 'IN19']
  0000201577DE803: ['CN8', 'IN19']
  0000201578DE7DD: ['CN8', 'IN19']
  0000201579DEB98: ['CN8', 'IN16']
  0000201580DE5DE: ['CN8', 'IN19']
  0000201764DE469: ['CN8', 'IN19']
  0000201765DE82C: ['CN8', 'IN19']
  0000201766DE4EC: ['CN8', 'IN19'

In [168]:
print(f"\nREGISTER-BASED APPLIANCE DETECTION PLAN")

# Define what to do with each register type for appliance fault detection
register_analysis_plan = {
    'IN19': {
        'description': 'Inclusive - Hot water control (19hrs available)',
        'appliances': ['Hot water heater (controlled)', 'Base load appliances (uncontrolled)'],
        'analysis': 'Identify controlled vs uncontrolled load patterns',
        'fault_detection': 'High baseload = stuck hot water heater, inverted day/night = control failure'
    },
    'CN8': {
        'description': 'Controlled - Heavy load control (8hrs available)', 
        'appliances': ['Heat pump', 'Electric heating', 'Major appliances'],
        'analysis': 'Analyze expected vs actual control periods',
        'fault_detection': 'Night operation = control failure, high baseload = stuck ON'
    },
    'IN16': {
        'description': 'Inclusive - Moderate control (16hrs available)',
        'appliances': ['Hot water heater (controlled)', 'Base load (uncontrolled)'],
        'analysis': 'Similar to IN19 but with more control',
        'fault_detection': 'High baseload = stuck hot water heater, pattern irregularities'
    },
    'UN24': {
        'description': 'Uncontrolled - No load management (24hrs)',
        'appliances': ['All appliances uncontrolled'],
        'analysis': 'Look for individual appliance signatures in total load',
        'fault_detection': 'High baseload = always-on faulty device, erratic patterns = multiple issues'
    },
    'N8|D16': {
        'description': 'Day/Night split meter',
        'appliances': ['Night: storage heating', 'Day: normal appliances'],
        'analysis': 'Compare day vs night consumption patterns',
        'fault_detection': 'Inverted patterns = timing control issues, high night load = heating faults'
    },
    'CN11': {
        'description': 'Controlled - Specific control (11hrs available)',
        'appliances': ['Specific controlled appliances'],
        'analysis': 'Analyze control compliance',
        'fault_detection': 'Off-period operation = control failure, high baseload = stuck device'
    },
    'CN16': {
        'description': 'Controlled - Moderate control (16hrs available)', 
        'appliances': ['Controlled appliances'],
        'analysis': 'Analyze control periods and compliance',
        'fault_detection': 'Off-period operation = control failure, pattern deviations = device issues'
    }
}

# Show analysis plan for registers found in our data
print(f"\nRegister types found in dataset:")
for register, info in register_analysis_plan.items():
    if register in unique_registers:
        count = register_data[register]['house_count']
        anomalous_count = len([h for h in register_data[register]['houses'] 
                              if h in all_houses_analysis['all_anomalous_houses']])
        
        print(f"\n{register} ({count} total houses, {anomalous_count} anomalous):")
        print(f"  Description: {info['description']}")
        print(f"  Expected appliances: {', '.join(info['appliances'])}")
        print(f"  Analysis approach: {info['analysis']}")
        print(f"  Fault detection: {info['fault_detection']}")


REGISTER-BASED APPLIANCE DETECTION PLAN

Register types found in dataset:

IN19 (96 total houses, 46 anomalous):
  Description: Inclusive - Hot water control (19hrs available)
  Expected appliances: Hot water heater (controlled), Base load appliances (uncontrolled)
  Analysis approach: Identify controlled vs uncontrolled load patterns
  Fault detection: High baseload = stuck hot water heater, inverted day/night = control failure

CN8 (26 total houses, 13 anomalous):
  Description: Controlled - Heavy load control (8hrs available)
  Expected appliances: Heat pump, Electric heating, Major appliances
  Analysis approach: Analyze expected vs actual control periods
  Fault detection: Night operation = control failure, high baseload = stuck ON

IN16 (18 total houses, 14 anomalous):
  Description: Inclusive - Moderate control (16hrs available)
  Expected appliances: Hot water heater (controlled), Base load (uncontrolled)
  Analysis approach: Similar to IN19 but with more control
  Fault detec

In [169]:
anomaly_groups = get_all_anomalous_houses()
all_three_methods_houses = list(anomaly_groups['breakdown']['all_three_methods'])
statistical_behavioral_houses = list(anomaly_groups['breakdown']['statistical_behavioral']) 
statistical_ml_houses = list(anomaly_groups['breakdown']['ml_statistical'])

METHOD 1 - ML Anomalies (15% contamination): 17 unique houses
METHOD 2 - Statistical Anomalies: 60 unique houses
METHOD 3 - Behavioral Pattern Anomalies: 34 unique houses
TOTAL UNIQUE ANOMALOUS HOUSES (ALL METHODS): 64

BREAKDOWN BY METHOD OVERLAP:
ML only:                      0 houses
Statistical only:            27 houses
Behavioral only:              4 houses
ML + Statistical:             3 houses
ML + Behavioural:              0 houses
Statistical + Behavioural:    16 houses
ALL THREE METHODS:           14 houses
APPLYING CROSS-METHOD VALIDATION
Requiring: Statistical Detection + (ML OR Behavioural confirmation)
Houses passing cross-validation: 33
Reduction: 64 → 33 houses

VALIDATED BREAKDOWN:
Statistical + ML:             3 houses
Statistical + Behavioral:    16 houses
ALL THREE METHODS:           14 houses


In [170]:
# SECTION 1: Calculate daily power metrics function
def calculate_daily_power_metrics(house_id, register, df, time_cols):
    # Get data for this house and register
    house_register_data = df[
        (df['ICP_IDENTIFIER'] == house_id) & 
        (df['REGISTER_CONTENTS'] == register)
    ].copy()
    
    if len(house_register_data) == 0:
        return pd.DataFrame()
    
    # Ensure datetime and add date column
    house_register_data['INTERVAL_DATE'] = pd.to_datetime(house_register_data['INTERVAL_DATE'])
    house_register_data['date'] = house_register_data['INTERVAL_DATE'].dt.date
    
    daily_metrics = []
    
    # Calculate metrics for each day
    for date, day_data in house_register_data.groupby('date'):
        if len(day_data) == 0:
            continue
            
        # Get power pattern for this specific day (should be 48 intervals)
        daily_pattern = day_data[time_cols].mean().values
        
        if len(daily_pattern) != 48:
            continue  # Skip incomplete days
        
        # Calculate power metrics
        avg_power = np.mean(daily_pattern)
        peak_power = np.max(daily_pattern)
        min_power = np.min(daily_pattern)
        
        # Estimated baseload (5th percentile)
        estimated_baseload = np.percentile(daily_pattern, 5)
        
        # Time period averages
        night_pattern = np.concatenate([daily_pattern[46:48], daily_pattern[0:12]])
        night_power = np.mean(night_pattern)
        morning_power = np.mean(daily_pattern[12:20])   # 06:00-10:00
        daytime_power = np.mean(daily_pattern[20:34])   # 10:00-17:00  
        evening_power = np.mean(daily_pattern[34:46])   # 17:00-23:00
        
        # Ratios for pattern analysis
        night_day_ratio = night_power / (daytime_power + 0.001)  # Avoid division by zero
        peak_to_baseload_ratio = peak_power / (estimated_baseload + 0.001)
        evening_to_night_ratio = evening_power / (night_power + 0.001)
        
        # Find peak hour (convert index to time)
        peak_hour_index = np.argmax(daily_pattern)
        peak_hour = peak_hour_index * 0.5  # Convert to hours (each interval is 0.5 hours)
        
        # Store daily metrics
        daily_metrics.append({
            'date': date,
            'house_id': house_id,
            'register': register,
            'avg_power': avg_power,
            'peak_power': peak_power,
            'min_power': min_power,
            'estimated_baseload': estimated_baseload,
            'morning_power': morning_power,
            'night_power': night_power,
            'day_power': daytime_power,
            'evening_power': evening_power,
            'night_day_ratio': night_day_ratio,
            'peak_to_baseload_ratio': peak_to_baseload_ratio,
            'evening_to_night_ratio': evening_to_night_ratio,
            'peak_hour': peak_hour,
            'power_range': peak_power - min_power
        })
    
    return pd.DataFrame(daily_metrics)

# SECTION 2: Calculate population thresholds
def calculate_population_thresholds(df, time_cols):
    
    # Get list of validated anomalous houses
    validated_anomalous_houses = set(all_houses_analysis['validated_anomalies'])
    all_houses = set(df['ICP_IDENTIFIER'].unique())
    normal_houses = list(all_houses - validated_anomalous_houses)
    
    print(f"Total houses: {len(all_houses)}")
    print(f"Validated anomalous houses: {len(validated_anomalous_houses)}")
    print(f"Normal houses for threshold calculation: {len(normal_houses)}")
    
    # Filter dataset to only include normal houses
    normal_df = df[df['ICP_IDENTIFIER'].isin(normal_houses)]
    print(f"Calculating thresholds from {len(normal_houses)} normal houses...")
    
    # Collect daily metrics from all normal houses
    all_daily_metrics = []
    houses_processed = 0
    houses_with_data = 0
    
    for house in normal_houses:
        house_data = normal_df[normal_df['ICP_IDENTIFIER'] == house]
        registers = house_data['REGISTER_CONTENTS'].unique()
        
        # Process each register for this house
        for register in registers:
            daily_metrics = calculate_daily_power_metrics(house, register, normal_df, time_cols)
            
            if len(daily_metrics) > 0:
                all_daily_metrics.append(daily_metrics)
                houses_with_data += 1
        
        houses_processed += 1
        if houses_processed % 20 == 0:
            print(f"  Processed {houses_processed}/{len(normal_houses)} normal houses...")
    
    # Combine all daily metrics
    if len(all_daily_metrics) > 0:
        population_daily_df = pd.concat(all_daily_metrics, ignore_index=True)
        print(f"Population data collected: {len(population_daily_df)} daily records from normal houses")
    else:
        print("ERROR: No daily metrics collected!")
        return None
    
    # Calculate population thresholds using percentiles
    thresholds = {
        # More sensitive thresholds (75th percentile)
        'baseload_75th': population_daily_df['estimated_baseload'].quantile(0.75),
        'night_power_75th': population_daily_df['night_power'].quantile(0.75),
        'day_power_75th': population_daily_df['day_power'].quantile(0.75),
        
        # Severe thresholds (95th percentile)
        'baseload_95th': population_daily_df['estimated_baseload'].quantile(0.95),
        'night_power_95th': population_daily_df['night_power'].quantile(0.95),
        'day_power_95th': population_daily_df['day_power'].quantile(0.95),
        
        # Pattern thresholds
        'night_day_ratio_95th': population_daily_df['night_day_ratio'].quantile(0.95),
        'night_power_50th': population_daily_df['night_power'].quantile(0.50),
        'night_day_ratio_75th': population_daily_df['night_day_ratio'].quantile(0.75),
        'power_range_75th': population_daily_df['power_range'].quantile(0.75),
        'night_power_25th': population_daily_df['night_power'].quantile(0.25),
    }
    
    print(f"  Population thresholds calculated successfully from NORMAL houses only!")
    print(f"   Baseload (75th percentile): {thresholds['baseload_75th']:.3f} kW")
    print(f"   Night power (75th percentile): {thresholds['night_power_75th']:.3f} kW")
    print(f"   Night/day ratio (75th percentile): {thresholds['night_day_ratio_75th']:.3f}")
    print(f"   Power range (75th percentile): {thresholds['power_range_75th']:.3f} kW")
    print("  Population thresholds calculation complete!")
    
    return thresholds

# Calculate the population thresholds using ONLY normal houses
print(" CALCULATING POPULATION THRESHOLDS FROM NORMAL HOUSES ONLY")
population_thresholds = calculate_population_thresholds(results['df'], results['time_cols'])

if population_thresholds:
    print(" Ready to run appliance fault analysis with accurate thresholds!")
else:
    print(" Failed to calculate population thresholds")

 CALCULATING POPULATION THRESHOLDS FROM NORMAL HOUSES ONLY
Total houses: 118
Validated anomalous houses: 33
Normal houses for threshold calculation: 85
Calculating thresholds from 85 normal houses...
  Processed 20/85 normal houses...
  Processed 40/85 normal houses...
  Processed 60/85 normal houses...
  Processed 80/85 normal houses...
Population data collected: 103327 daily records from normal houses
  Population thresholds calculated successfully from NORMAL houses only!
   Baseload (75th percentile): 0.894 kW
   Night power (75th percentile): 1.903 kW
   Night/day ratio (75th percentile): 1.078
   Power range (75th percentile): 4.880 kW
  Population thresholds calculation complete!
 Ready to run appliance fault analysis with accurate thresholds!


In [171]:
def detect_primary_fault_with_explanation(house_id, register, daily_metrics, thresholds, feature_df):
    """
    Improved fault detection with register-specific logic
    """
    
    if len(daily_metrics) == 0:
        return None
    
    # Get house characteristics for context
    house_features = feature_df.loc[house_id] if house_id in feature_df.index else None
    
    # Define thresholds (make them more stringent)
    baseload_threshold = thresholds['baseload_95th']  # Use 95th percentile for more serious faults
    night_threshold = thresholds['night_power_95th'] 
    ratio_threshold = thresholds['night_day_ratio_95th']
    range_threshold = thresholds['power_range_75th']
    
    # Calculate fault statistics
    high_baseload_days = daily_metrics[daily_metrics['estimated_baseload'] > baseload_threshold]
    high_night_days = daily_metrics[daily_metrics['night_power'] > night_threshold]
    inverted_days = daily_metrics[daily_metrics['night_day_ratio'] > ratio_threshold]
    erratic_days = daily_metrics[daily_metrics['power_range'] > range_threshold]
    
    total_days = len(daily_metrics)
    
    # Count fault percentages
    baseload_pct = len(high_baseload_days) / total_days * 100
    night_pct = len(high_night_days) / total_days * 100
    inverted_pct = len(inverted_days) / total_days * 100
    erratic_pct = len(erratic_days) / total_days * 100
    
    # REGISTER-SPECIFIC FAULT LOGIC
    primary_fault_type = None
    primary_fault_pct = 0
    fault_days = pd.DataFrame()
    
    # Define register-specific fault priorities and thresholds
    register_fault_logic = {
        'IN19': {
            'primary_candidates': ['baseload', 'inverted_pattern'],  # Hot water issues most common
            'secondary_candidates': ['night_operation', 'erratic_behavior'],
            'min_threshold': 15  # Must affect >15% of days to be significant
        },
        'CN8': {
            'primary_candidates': ['baseload', 'night_operation'],  # Control system issues
            'secondary_candidates': ['inverted_pattern', 'erratic_behavior'], 
            'min_threshold': 10  # Controlled systems should be more stable
        },
        'IN16': {
            'primary_candidates': ['baseload', 'inverted_pattern'],  # Similar to IN19
            'secondary_candidates': ['night_operation', 'erratic_behavior'],
            'min_threshold': 15
        },
        'UN24': {
            'primary_candidates': ['baseload', 'erratic_behavior'],  # No control system to invert
            'secondary_candidates': ['night_operation'],  # inverted_pattern should be rare
            'min_threshold': 20  # Higher threshold for uncontrolled systems
        },
        'N8|D16': {
            'primary_candidates': ['inverted_pattern', 'baseload'],  # Day/night timing critical
            'secondary_candidates': ['night_operation', 'erratic_behavior'],
            'min_threshold': 10  # Timing systems should be precise
        },
        'CN11': {
            'primary_candidates': ['baseload', 'night_operation'],  # Similar to CN8
            'secondary_candidates': ['inverted_pattern', 'erratic_behavior'],
            'min_threshold': 10
        },
        'CN16': {
            'primary_candidates': ['baseload', 'night_operation'],  # Similar to other controlled
            'secondary_candidates': ['inverted_pattern', 'erratic_behavior'],
            'min_threshold': 10
        }
    }
    
    # Get register-specific logic (default to UN24 if not found)
    logic = register_fault_logic.get(register, register_fault_logic['UN24'])
    
    # All fault scores
    fault_scores = {
        'baseload': baseload_pct,
        'night_operation': night_pct,
        'inverted_pattern': inverted_pct,
        'erratic_behavior': erratic_pct
    }
    
    print(f"   Fault scores: baseload={baseload_pct:.1f}%, night={night_pct:.1f}%, inverted={inverted_pct:.1f}%, erratic={erratic_pct:.1f}%")
    print(f"   Register {register} logic: primary={logic['primary_candidates']}, min_threshold={logic['min_threshold']}%")
    
    # First, check primary candidates
    for fault_type in logic['primary_candidates']:
        if fault_scores[fault_type] >= logic['min_threshold']:
            if fault_scores[fault_type] > primary_fault_pct:
                primary_fault_type = fault_type
                primary_fault_pct = fault_scores[fault_type]
    
    # If no primary fault found, check secondary candidates with higher threshold
    if primary_fault_type is None:
        higher_threshold = logic['min_threshold'] + 10  # Require higher percentage for secondary faults
        for fault_type in logic['secondary_candidates']:
            if fault_scores[fault_type] >= higher_threshold:
                if fault_scores[fault_type] > primary_fault_pct:
                    primary_fault_type = fault_type
                    primary_fault_pct = fault_scores[fault_type]
    
    # If still no fault found, check if any fault exceeds absolute minimum (5%)
    if primary_fault_type is None:
        absolute_min = 5
        for fault_type, pct in fault_scores.items():
            if pct >= absolute_min and pct > primary_fault_pct:
                primary_fault_type = fault_type
                primary_fault_pct = pct
    
    # If no significant fault, return None
    if primary_fault_type is None:
        return None
    
    print(f"   Selected fault: {primary_fault_type} ({primary_fault_pct:.1f}%)")
    
    # Get the fault days for the primary issue
    if primary_fault_type == 'baseload':
        fault_days = high_baseload_days
        metric_column = 'estimated_baseload'
        threshold_used = baseload_threshold
    elif primary_fault_type == 'night_operation':
        fault_days = high_night_days
        metric_column = 'night_power'
        threshold_used = night_threshold
    elif primary_fault_type == 'inverted_pattern':
        fault_days = inverted_days
        metric_column = 'night_day_ratio'
        threshold_used = ratio_threshold
    else:  # erratic_behavior
        fault_days = erratic_days
        metric_column = 'power_range'
        threshold_used = range_threshold
    
    # Analyze fault timeline
    fault_timeline = analyze_fault_timeline(fault_days, daily_metrics, primary_fault_type)
    
    # Generate explanation based on register type and fault
    explanation = generate_fault_explanation(register, primary_fault_type, house_features, fault_timeline)
    
    return {
        'house_id': house_id,
        'register': register,
        'primary_fault_type': primary_fault_type,
        'fault_percentage': primary_fault_pct,
        'fault_days_count': len(fault_days),
        'total_days': total_days,
        'threshold_used': threshold_used,
        'metric_column': metric_column,
        'fault_timeline': fault_timeline,
        'explanation': explanation,
        'all_fault_scores': fault_scores,  
        'house_characteristics': {
            'rooms': house_features['num_rooms'] if house_features is not None else 'Unknown',
            'avg_power_per_room': house_features['avg_power_per_room'] if house_features is not None else 'Unknown',
            'baseload': house_features['baseload_kW'] if house_features is not None else 'Unknown',
            'load_factor': house_features['load_factor'] if house_features is not None else 'Unknown',
            'night_usage_ratio': house_features['night_usage_ratio'] if house_features is not None else 'Unknown'
        }
    }

def analyze_fault_timeline(fault_days, daily_metrics, fault_type):
    if len(fault_days) == 0:
        return {'status': 'NO_FAULT'}
    
    # Sort by date
    fault_days_sorted = fault_days.sort_values('date')
    all_days_sorted = daily_metrics.sort_values('date')
    
    # Timeline info
    fault_start = fault_days_sorted['date'].min()
    fault_end = fault_days_sorted['date'].max()
    dataset_end = all_days_sorted['date'].max()
    dataset_start = all_days_sorted['date'].min()
    
    # Calculate days since last fault
    days_since_last_fault = (dataset_end - fault_end).days
    
    # Check recent activity (last 30 days)
    recent_cutoff = dataset_end - pd.Timedelta(days=30)
    recent_data = all_days_sorted[all_days_sorted['date'] >= recent_cutoff]
    recent_fault_days = fault_days_sorted[fault_days_sorted['date'] >= recent_cutoff]
    
    recent_fault_percentage = (len(recent_fault_days) / len(recent_data) * 100) if len(recent_data) > 0 else 0
    
    # Check if data ends before Sept 2024 (indicating early data cutoff)
    data_cutoff_date = pd.to_datetime('2024-08-01')  # Conservative cutoff
    has_recent_data = dataset_end >= data_cutoff_date.date()
    
    # Determine status 
    if not has_recent_data:
        # Special handling for houses with early data cutoffs
        if days_since_last_fault <= 7:
            status = 'UNRESOLVED_AT_CUTOFF'
            severity = 'UNKNOWN'
        else:
            status = 'RESOLVED_AT_CUTOFF'
            severity = 'UNKNOWN'
    elif recent_fault_percentage > 20:  # >20% of recent days
        status = 'ONGOING'
        severity = 'HIGH'
    elif days_since_last_fault <= 7:
        status = 'ONGOING'
        severity = 'HIGH'
    elif days_since_last_fault <= 30:
        if recent_fault_percentage > 10:
            status = 'INTERMITTENT'
            severity = 'MEDIUM'
        else:
            status = 'RECENTLY_RESOLVED'
            severity = 'LOW'
    elif days_since_last_fault <= 90:
        status = 'RECENTLY_RESOLVED'
        severity = 'LOW'
    elif days_since_last_fault <= 365:  # Within last year
        status = 'RESOLVED'
        severity = 'LOW'
    else:  # More than 1 year ago
        status = 'RESOLVED_LONG_AGO'
        severity = 'LOW'
    
    # Find continuous fault periods
    continuous_periods = find_continuous_fault_periods(fault_days_sorted)
    
    return {
        'status': status,
        'severity': severity,
        'fault_start': fault_start,
        'fault_end': fault_end,
        'days_since_last_fault': days_since_last_fault,
        'recent_fault_percentage': recent_fault_percentage,
        'continuous_periods': continuous_periods,
        'total_fault_days': len(fault_days),
        'dataset_span_days': (dataset_end - dataset_start).days
    }

def find_continuous_fault_periods(fault_days_sorted):
    #Find continuous periods of faults (3+ consecutive days)
    
    if len(fault_days_sorted) == 0:
        return []
    
    periods = []
    current_start = fault_days_sorted.iloc[0]['date']
    current_end = fault_days_sorted.iloc[0]['date']
    current_count = 1
    
    for i in range(1, len(fault_days_sorted)):
        prev_date = fault_days_sorted.iloc[i-1]['date']
        curr_date = fault_days_sorted.iloc[i]['date']
        
        # Check if consecutive (within 2 days to allow for missing data)
        if (curr_date - prev_date).days <= 2:
            current_end = curr_date
            current_count += 1
        else:
            # End of continuous period
            if current_count >= 7:  # Only include periods of 7+ days
                periods.append({
                    'start_date': current_start,
                    'end_date': current_end,
                    'duration_days': current_count
                })
            
            # Start new period
            current_start = curr_date
            current_end = curr_date
            current_count = 1
    
    # The last period
    if current_count >= 3:
        periods.append({
            'start_date': current_start,
            'end_date': current_end,
            'duration_days': current_count
        })
    
    return periods



def generate_fault_explanation(register, fault_type, house_features, fault_timeline):
    
    explanations = {
    'IN19': {
        'baseload': {
            'appliance': 'Likely Hot Water Cylinder',
            'fault': 'Thermostat likely stuck or heating element possibly always ON',
            'cause': 'The hot water heating element is likely not switching off properly, causing continuous power draw even when water is probably hot.',
            'action': 'Check cylinder thermostat, heating elements, and temperature sensors. May need thermostat replacement.'
        },
        'inverted_pattern': {
            'appliance': 'Possible Hot Water Control System',
            'fault': 'Load control timing likely malfunctioning',
            'cause': 'The ripple control system is probably not properly managing when the hot water heater operates, causing it to run during peak periods.',
            'action': 'Inspect ripple control receiver, check control signal reception, verify time-of-use programming.'
        },
        'night_operation': {
            'appliance': 'Likely Hot Water Heater',
            'fault': 'Excessive night heating cycles probable',
            'cause': 'Hot water system is likely drawing more power than normal during controlled periods, indicating possible oversized elements or heat loss.',
            'action': 'Check for cylinder heat loss, inspect insulation, verify element sizing and thermostat settings.'
        },
        'erratic_behavior': {
            'appliance': 'Likely Hot Water System',
            'fault': 'Hot water system probably cycling erratically',
            'cause': 'Hot water heating system is likely experiencing control issues, thermostat problems, or electrical faults causing irregular operation.',
            'action': 'Inspect hot water control system, check thermostat operation, verify electrical connections and heating element function.'
        }
    },
    'CN8': {
        'baseload': {
            'appliance': 'Likely Heat Pump or Heavy Load Device',
            'fault': 'Device probably stuck in ON position',
            'cause': 'The controlled device contactor or control board is likely faulty, preventing the system from switching off when not needed.',
            'action': 'Inspect device control board, check contactors, verify control signal operation.'
        },
        'inverted_pattern': {
            'appliance': 'Likely Controlled Heavy Load System',
            'fault': 'Control timing pattern probably inverted',
            'cause': 'The 8-hour control system is likely operating at wrong times, possibly running during peak periods instead of off-peak.',
            'action': 'Check load control timing, verify 8-hour availability window, inspect control signal scheduling.'
        },
        'night_operation': {
            'appliance': 'Possible Heat Pump or Controlled Device',
            'fault': 'Device likely running outside control hours',
            'cause': 'The load control system is probably not properly restricting device operation to designated hours (typically 8 hours).',
            'action': 'Check ripple control receiver, verify control signal, inspect device control interface.'
        },
        'erratic_behavior': {
            'appliance': 'Likely Heat Pump or Heavy Load System',
            'fault': 'Device probably short-cycling or irregular operation',
            'cause': 'Faulty thermostat, possible efficiency issues, or electrical problems likely causing the device to cycle on/off erratically.',
            'action': 'Inspect device control system, check efficiency parameters, verify electrical connections and control logic.'
        }
    },
    'CN11': {
        'baseload': {
            'appliance': 'Likely Controlled Heating System',
            'fault': 'Equipment probably not switching off when control signal sent',
            'cause': 'The controlled device is likely not responding to control signals, causing continuous operation.',
            'action': 'Check control signal reception, inspect device control interface, verify switching mechanisms.'
        },
        'inverted_pattern': {
            'appliance': 'Likely Controlled Device System',
            'fault': 'Control timing pattern probably inverted',
            'cause': 'The 11-hour control system is likely operating at incorrect times, possibly during restricted periods.',
            'action': 'Check control timing schedule, verify 11-hour availability window, inspect timing signal accuracy.'
        },
        'night_operation': {
            'appliance': 'Possible Controlled Device',
            'fault': 'Device likely operating outside designated control hours',
            'cause': 'Load management system is probably not restricting operation to the designated 11-hour window.',
            'action': 'Inspect load management system, verify control timing, check device compliance.'
        },
        'erratic_behavior': {
            'appliance': 'Likely Controlled Equipment',
            'fault': 'Control system timing issues probable',
            'cause': 'The 11-hour control system is likely experiencing timing or switching malfunctions.',
            'action': 'Check control system timing, inspect switching equipment, verify signal integrity.'
        }
    },
    'CN16': {
        'baseload': {
            'appliance': 'Likely Controlled Heating/Cooling System',
            'fault': 'Equipment probably not responding to control signals',
            'cause': 'The controlled device is likely stuck ON and not responding to network load management signals.',
            'action': 'Inspect control signal reception, check device interface, verify switching operation.'
        },
        'inverted_pattern': {
            'appliance': 'Likely Controlled System',
            'fault': 'Control timing pattern probably inverted',
            'cause': 'The 16-hour control system is likely operating during wrong periods, possibly during the 8-hour restricted window.',
            'action': 'Check control scheduling, verify 16-hour availability operation, inspect timing control system.'
        },
        'night_operation': {
            'appliance': 'Possible Controlled Major Appliance',
            'fault': 'Device likely operating during restricted 8-hour control period',
            'cause': 'Load management system is probably not properly restricting operation during peak control periods.',
            'action': 'Check load management timing, verify 16-hour availability window operation.'
        },
        'erratic_behavior': {
            'appliance': 'Likely Controlled System',
            'fault': 'Control system or equipment likely malfunctioning',
            'cause': 'Either the control system or the controlled equipment is probably experiencing operational issues.',
            'action': 'Comprehensive inspection of control system and controlled equipment.'
        }
    },
    'IN16': {
        'baseload': {
            'appliance': 'Likely Hot Water Cylinder with Controlled Element',
            'fault': 'Controlled heating element probably stuck ON or thermostat failure',
            'cause': 'The controlled portion (likely hot water heating) is probably not switching off properly while base load continues.',
            'action': 'Check controlled element thermostat, heating elements, and control system. Inspect base load for always-on device issues.'
        },
        'inverted_pattern': {
            'appliance': 'Possible Load Control System',
            'fault': 'Load control timing likely malfunctioning',
            'cause': 'The 16-hour control system is probably not managing controlled loads properly, affecting timing patterns.',
            'action': 'Inspect ripple control system, verify 16-hour control window operation, check timing signals.'
        },
        'night_operation': {
            'appliance': 'Likely Controlled Heating Device',
            'fault': 'Controlled device probably operating excessively during night periods',
            'cause': 'The controlled portion is likely drawing more power than normal during controlled periods, indicating possible oversizing or efficiency issues.',
            'action': 'Check controlled device efficiency, inspect element sizing, verify control period operation.'
        },
        'erratic_behavior': {
            'appliance': 'Likely Multiple Systems',
            'fault': 'Multiple device malfunctions probable',
            'cause': 'Several appliances are likely operating erratically, possibly due to electrical issues, faulty controls, or aging equipment.',
            'action': 'Comprehensive electrical inspection, check all major appliances, verify supply voltage stability.'
        }
    },
    'N8|D16': {
        'baseload': {
            'appliance': 'Likely Storage Heating Elements',
            'fault': 'Heating elements probably stuck ON',
            'cause': 'Storage heating elements are likely not switching off after night charging period, causing continuous power draw.',
            'action': 'Check storage heater control systems, inspect heating elements and contactors, verify timer operation.'
        },
        'inverted_pattern': {
            'appliance': 'Likely Storage Heating System',
            'fault': 'Day/night control timing probably failing',
            'cause': 'The storage heating control system is likely not properly managing when heating elements charge (should be night) vs when heat is released (day).',
            'action': 'Inspect storage heater control timers, check day/night rate switching, verify element operation and charging cycles.'
        },
        'night_operation': {
            'appliance': 'Possible Storage Heating',
            'fault': 'Excessive night charging likely',
            'cause': 'Storage heaters are probably drawing more power than normal during night periods, indicating possible oversized elements or control issues.',
            'action': 'Verify storage heater element sizing, check control settings, inspect for heat loss issues.'
        },
        'erratic_behavior': {
            'appliance': 'Likely Day/Night Control System',
            'fault': 'Day/night switching system probably malfunctioning',
            'cause': 'The day/night switching control is likely experiencing timing issues or switching faults, causing erratic load patterns.',
            'action': 'Inspect day/night switching system, check timing controls, verify switching mechanism operation.'
        }
    },
    'UN24': {
        'baseload': {
            'appliance': 'Likely Always-On Device',
            'fault': 'Faulty appliance probably running continuously',
            'cause': 'An appliance (likely heating, hot water, or motor) is probably stuck in the ON position, creating abnormally high constant power draw.',
            'action': 'Inspect all major appliances for continuous operation, check thermostats, timers, and control systems.'
        },
        'inverted_pattern': {
            'appliance': 'Likely Household Load Pattern',
            'fault': 'Usage pattern probably inverted from normal',
            'cause': 'Household usage patterns are likely inverted, possibly due to occupancy changes, faulty timers, or unusual appliance operation schedules.',
            'action': 'Check occupancy patterns, inspect appliance timers, verify normal household operation schedules.'
        },
        'night_operation': {
            'appliance': 'Likely Night-time Appliances',
            'fault': 'Excessive night-time power consumption probable',
            'cause': 'Appliances are likely consuming more power than normal during night hours, indicating possible heating, cooling, or other device issues.',
            'action': 'Inspect heating/cooling systems, check water heating, verify other major appliance night operation.'
        },
        'erratic_behavior': {
            'appliance': 'Likely Multiple Appliances',
            'fault': 'Multiple device malfunctions probable',
            'cause': 'Several appliances are likely operating erratically, possibly due to electrical issues, faulty controls, or aging equipment.',
            'action': 'Comprehensive electrical inspection, check all major appliances, verify supply voltage stability.'
        }
    }
}
    # Get explanation for this register/fault combination
    register_explanations = explanations.get(register)
    fault_explanation = register_explanations.get(fault_type)
    
    # Add status and timeline context
    status_emoji = {
        'ONGOING': '🔴',
        'INTERMITTENT': '🟡', 
        'RECENTLY_RESOLVED': '🟢',
        'RESOLVED': '✅',
        'RESOLVED_LONG_AGO': '✅',
        'UNRESOLVED_AT_CUTOFF': '⚠️',
        'RESOLVED_AT_CUTOFF': '📊'
    }
    
    severity_emoji = {
        'HIGH': '🚨',
        'MEDIUM': '⚠️',
        'LOW': '✓'
    }
    
    status = fault_timeline['status']
    severity = fault_timeline['severity']
    
    # Build comprehensive explanation
    explanation = {
        'status_emoji': status_emoji.get(status, '❓'),
        'severity_emoji': severity_emoji.get(severity, '❓'),
        'appliance': fault_explanation['appliance'],
        'fault_description': fault_explanation['fault'],
        'likely_cause': fault_explanation['cause'],
        'recommended_action': fault_explanation['action'],
        'current_status': status,
        'severity': severity,
        'fault_period': f"{fault_timeline['fault_start']} to {fault_timeline['fault_end']}",
        'days_since_last_fault': fault_timeline['days_since_last_fault']
    }
    
    return explanation

def run_enhanced_appliance_analysis(house_list, df, time_cols, thresholds, feature_df):
    
    print("APPLIANCE FAULT ANALYSIS")
    print(f"Analyzing {len(house_list)} anomalous houses")
    
    results = []
    
    for i, house_id in enumerate(house_list):
        print(f"\n[{i+1}/{len(house_list)}] House: {house_id}")
    
        
        # Get house data and registers
        house_data = df[df['ICP_IDENTIFIER'] == house_id]
        registers = house_data['REGISTER_CONTENTS'].unique()
        
        house_results = {
            'house_id': house_id,
            'registers': list(registers),
            'faults': []
        }
        
        # Analyze each register
        for register in registers:
            print(f"Register: {register}")
            
            # Get register description
            register_descriptions = {
                'IN19': 'Inclusive 19-hour control (Hot water heating + base load)',
                'CN8': 'Controlled 8-hour supply (Heavy controlled load)',
                'IN16': 'Inclusive 16-hour control (Mixed controlled/uncontrolled loads)',
                'UN24': 'Uncontrolled 24-hour supply (All household loads)',
                'N8|D16': 'Day/Night split meter (Night 8hr, Day 16hr)',
                'CN11': 'Controlled 11-hour supply (Moderate controlled load)',
                'CN16': 'Controlled 16-hour supply (Light controlled load)'
            }
            
            register_desc = register_descriptions.get(register, 'Unknown register type')
            print(f"   Type: {register_desc}")
            
            # Calculate daily metrics
            daily_metrics = calculate_daily_power_metrics(house_id, register, df, time_cols)
            
            if len(daily_metrics) == 0:
                print("   No data available")
                continue
            
            print(f"   Monitoring period: {len(daily_metrics)} days")
            print(f"   Date range: {daily_metrics['date'].min()} to {daily_metrics['date'].max()}")
            
            # Detect primary fault
            fault_analysis = detect_primary_fault_with_explanation(
                house_id, register, daily_metrics, thresholds, feature_df
            )
            
            if fault_analysis is None:
                print("   No significant faults detected")
                print("   System appears to be operating within normal parameters")
                continue
            
            # Display results with enhanced formatting
            explanation = fault_analysis['explanation']
            timeline = fault_analysis['fault_timeline']
            
            print(f"\n   {explanation['status_emoji']} {explanation['severity_emoji']} PRIMARY FAULT DETECTED")
            print(f"   Suspected Device: {explanation['appliance']}")
            print(f"   Fault Type: {explanation['fault_description']}")
            print(f"   Impact: {fault_analysis['fault_days_count']} days affected ({fault_analysis['fault_percentage']:.1f}% of monitoring period)")
            print(f"   Fault Period: {explanation['fault_period']}")
            print(f"   Current Status: {explanation['current_status']}")
            
            # Status explanation
            status_explanations = {
                'ONGOING': 'Fault is currently active and detected recently',
                'INTERMITTENT': 'Fault occurs sporadically with recent activity',
                'RECENTLY_RESOLVED': 'Fault appears to have been fixed recently',
                'RESOLVED': 'Fault was resolved some time ago',
                'RESOLVED_LONG_AGO': 'Fault was resolved over a year ago',
                'UNRESOLVED_AT_CUTOFF': 'Fault status unclear due to data cutoff',
                'RESOLVED_AT_CUTOFF': 'Fault appears resolved but data ends early'
            }
            
            status_exp = status_explanations.get(explanation['current_status'], 'Status unknown')
            print(f"   Status Details: {status_exp}")
            print(f"   Days Since Last Fault: {explanation['days_since_last_fault']} days")
            
            # Show continuous fault periods if any (limit to most significant ones)
            if timeline['continuous_periods']:
                significant_periods = [p for p in timeline['continuous_periods'] if p['duration_days'] >= 7]
                if significant_periods:
                    print(f"   Major Continuous Fault Periods:")
                    for period in significant_periods:  
                        print(f"      • {period['start_date']} to {period['end_date']} ({period['duration_days']} days)")
                    
            
            print(f"\n   Likely Cause:")
            print(f"      {explanation['likely_cause']}")
            print(f"    Recommended Action:")
            print(f"      {explanation['recommended_action']}")
            
            # Add severity assessment
            severity_descriptions = {
                'HIGH': 'Immediate attention required - fault is ongoing or frequent',
                'MEDIUM': 'Monitor closely - intermittent issues detected',
                'LOW': 'Low priority - fault appears resolved or infrequent',
                'UNKNOWN': 'Severity unclear due to limited recent data'
            }
            severity_desc = severity_descriptions.get(explanation['severity'], 'Severity unknown')
            print(f"    Priority Level: {explanation['severity']} - {severity_desc}")
            
            # Add house characteristics for context
            characteristics = fault_analysis['house_characteristics']
            print(f"\n    House Context:")
            print(f"      • Rooms: {characteristics['rooms']}")
            if isinstance(characteristics['avg_power_per_room'], (int, float)):
                print(f"      • Average power per room: {characteristics['avg_power_per_room']:.2f} kW")
            if isinstance(characteristics['baseload'], (int, float)):
                print(f"      • Typical baseload: {characteristics['baseload']:.2f} kW")
            
            # Add to results
            house_results['faults'].append(fault_analysis)
        
        results.append(house_results)
    
    # Summary statistics
    print("ANALYSIS SUMMARY")
    
    total_houses = len(house_list)
    houses_with_faults = len([r for r in results if r['faults']])
    total_faults = sum(len(r['faults']) for r in results)
    
    print(f" Houses analysed: {total_houses}")
    print(f" Houses with detected faults: {houses_with_faults}")
    print(f" Total faults detected: {total_faults}")
    
    # Fault type breakdown
    fault_types = {}
    fault_statuses = {}
    
    for house_result in results:
        for fault in house_result['faults']:
            fault_type = fault['primary_fault_type']
            fault_status = fault['explanation']['current_status']
            
            fault_types[fault_type] = fault_types.get(fault_type, 0) + 1
            fault_statuses[fault_status] = fault_statuses.get(fault_status, 0) + 1
    
    if fault_types:
        print(f"\n Fault Types:")
        for fault_type, count in sorted(fault_types.items(), key=lambda x: x[1], reverse=True):
            print(f"   • {fault_type.replace('_', ' ').title()}: {count} instances")
    
    if fault_statuses:
        print(f"\n Fault Status Distribution:")
        for status, count in sorted(fault_statuses.items(), key=lambda x: x[1], reverse=True):
            print(f"   • {status.replace('_', ' ').title()}: {count} faults")
    
    return results

In [172]:
all_three_results = run_enhanced_appliance_analysis(
    all_three_methods_houses, 
    results['df'], 
    results['time_cols'], 
    population_thresholds, 
    results['feature_df']
)

APPLIANCE FAULT ANALYSIS
Analyzing 14 anomalous houses

[1/14] House: 0000003686DECCA
Register: IN19
   Type: Inclusive 19-hour control (Hot water heating + base load)
   Monitoring period: 971 days
   Date range: 2022-01-01 to 2024-09-02
   Fault scores: baseload=42.9%, night=39.0%, inverted=6.9%, erratic=44.9%
   Register IN19 logic: primary=['baseload', 'inverted_pattern'], min_threshold=15%
   Selected fault: baseload (42.9%)

   🔴 🚨 PRIMARY FAULT DETECTED
   Suspected Device: Likely Hot Water Cylinder
   Fault Type: Thermostat likely stuck or heating element possibly always ON
   Impact: 417 days affected (42.9% of monitoring period)
   Fault Period: 2022-02-27 to 2024-09-02
   Current Status: ONGOING
   Status Details: Fault is currently active and detected recently
   Days Since Last Fault: 0 days
   Major Continuous Fault Periods:
      • 2022-05-17 to 2022-05-31 (15 days)
      • 2022-06-04 to 2022-06-19 (16 days)
      • 2022-07-10 to 2022-08-25 (43 days)
      • 2022-09-04 t

In [173]:
statistical_behavioral_results = run_enhanced_appliance_analysis(
    statistical_behavioral_houses,
    results['df'], 
    results['time_cols'], 
    population_thresholds, 
    results['feature_df']
)

APPLIANCE FAULT ANALYSIS
Analyzing 16 anomalous houses

[1/16] House: 0000002805DE2EB
Register: IN19
   Type: Inclusive 19-hour control (Hot water heating + base load)
   Monitoring period: 971 days
   Date range: 2022-01-01 to 2024-09-02
   Fault scores: baseload=5.0%, night=2.8%, inverted=0.9%, erratic=79.5%
   Register IN19 logic: primary=['baseload', 'inverted_pattern'], min_threshold=15%
   Selected fault: erratic_behavior (79.5%)

   🔴 🚨 PRIMARY FAULT DETECTED
   Suspected Device: Likely Hot Water System
   Fault Type: Hot water system probably cycling erratically
   Impact: 772 days affected (79.5% of monitoring period)
   Fault Period: 2022-01-12 to 2024-08-30
   Current Status: ONGOING
   Status Details: Fault is currently active and detected recently
   Days Since Last Fault: 3 days
   Major Continuous Fault Periods:
      • 2022-03-13 to 2022-06-27 (103 days)
      • 2022-06-30 to 2022-11-12 (134 days)
      • 2022-11-15 to 2022-12-10 (18 days)
      • 2023-02-11 to 2023-12-

In [174]:
statistical_ml_results = run_enhanced_appliance_analysis(
    statistical_ml_houses,
    results['df'], 
    results['time_cols'], 
    population_thresholds, 
    results['feature_df']
)

APPLIANCE FAULT ANALYSIS
Analyzing 3 anomalous houses

[1/3] House: 0000003251DE043
Register: IN19
   Type: Inclusive 19-hour control (Hot water heating + base load)
   Monitoring period: 588 days
   Date range: 2023-01-21 to 2024-09-02
   Fault scores: baseload=0.0%, night=0.0%, inverted=5.8%, erratic=32.5%
   Register IN19 logic: primary=['baseload', 'inverted_pattern'], min_threshold=15%
   Selected fault: erratic_behavior (32.5%)

   🔴 🚨 PRIMARY FAULT DETECTED
   Suspected Device: Likely Hot Water System
   Fault Type: Hot water system probably cycling erratically
   Impact: 191 days affected (32.5% of monitoring period)
   Fault Period: 2023-02-23 to 2024-08-30
   Current Status: ONGOING
   Status Details: Fault is currently active and detected recently
   Days Since Last Fault: 3 days
   Major Continuous Fault Periods:
      • 2023-05-07 to 2023-05-16 (7 days)
      • 2023-07-24 to 2023-08-16 (22 days)
      • 2023-08-27 to 2023-09-04 (7 days)
      • 2023-09-14 to 2023-09-22 (7 